# Get combined mechanisms data

In [1]:
import numpy, os
import pandas as pd

## Read the data

In [2]:
datasets = [{"name": "exact_recall",
             "queries_file": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_exact.json",
             "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/causal_trace_2377617/cases",
             "filename_template": "knowledge_{}_mlp.npz" 
            },
              {"name": "prompt_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/prompt_bias_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/prompt_bias_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "person_name_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/person_name_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/person_name_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "guesswork",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_guesswork.json",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/causal_trace_guesswork_2388522/cases",
              "filename_template": "knowledge_{}_mlp.npz"
              }
             ]

data_shares = {"exact_recall": 0.33,
               "prompt_bias": 0.17,
               "person_name_bias": 0.17,
               "guesswork": 0.33
              }
assert sum(data_shares.values())==1

num_samples = 1000

In [3]:
data = pd.DataFrame()
for dataset in datasets:
    tmp_data = pd.read_json(dataset["queries_file"], lines=dataset["queries_file"].endswith("jsonl")).sample(
        int(num_samples*data_shares[dataset["name"]]), random_state=42)
    tmp_data["type"] = dataset["name"]
    tmp_data["CT_results_dir"] = dataset["CT_results_dir"]
    tmp_data["filename_template"] = dataset["filename_template"]
    data = pd.concat((data, tmp_data), ignore_index=True)
    
data

,attribute,subject,relation_id,source,prompt,template,sub_view_rates,obj_view_rates,prediction,p_answers,...,answers_for_PB_She,confidence,candidate_prediction,candidate_p,top10_tokens,prediction_p,confident_flag,answers_for_PB_It,answers_for_PB_The_city,answers_for_PB_The_organisation
0,Paris,Chanel,P740,TREx_UHN,Chanel was formed in,[X] was formed in [Y],82295.083333,227647.083333,Paris,0.053301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tennessee,Nashville,P1376,TREx_UHN,Nashville is the capital of,[X] is the capital of [Y],9305.666667,99814.250000,Tennessee,0.479532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Austria,Vienna,P1376,TREx_UHN,Vienna is the capital of,[X] is the capital of [Y],123738.166667,217938.833333,Austria,0.563785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,France,The Fifth Element,P495,TREx_UHN,The Fifth Element was created in,[X] was created in [Y],73525.000000,424490.500000,France,0.181571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lithuania,Vilnius,P1376,TREx_UHN,Vilnius is the capital city of,[X] is the capital city of [Y],60167.750000,164579.833333,Lithuania,0.611381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Carlisle,Samuel Sterett,P19,Google_RE_UHN,Samuel Sterett was born in,[X] was born in [Y],83.416667,15319.000000,London,0.042578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,Sydney,Danielle McGrath,P19,Google_RE_UHN,Danielle McGrath is originally from,[X] is originally from [Y],100.416667,131092.166667,Ireland,0.014676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,Edmonton,Judd Buchanan,P19,Google_RE_UHN,Judd Buchanan was born in,[X] was born in [Y],195.000000,61414.916667,London,0.024318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,Edinburgh,William Erskine,P19,Google_RE_UHN,William Erskine was originally from,[X] was originally from [Y],53.750000,111502.666667,England,0.0153,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.type.value_counts()

exact_recall        330
guesswork           330
prompt_bias         170
person_name_bias    170
Name: type, dtype: int64

Save the data

In [5]:
data.to_json("/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_combined_mechanisms.json", orient="records")